# Retrieval

Retrieval is the centerpiece of our retrieval augmented generation (RAG) flow. 

Let's get our vectorDB from before.

## Vectorstore retrieval


In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
#!pip install lark

### Similarity Search

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'data/chroma/'

In [6]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [8]:
print(vectordb._collection.count())

209


In [9]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [10]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [11]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [12]:
smalldb.similarity_search(question, k=2)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).')]

In [13]:
smalldb.max_marginal_relevance_search(question, k=2, fetch_k=3)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

### Addressing Diversity: Maximum marginal relevance

Last class we introduced one problem: how to enforce diversity in the search results.
 
`Maximum marginal relevance` strives to achieve both relevance to the query *and diversity* among the results.

In [14]:
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question,k=3)

In [15]:
docs_ss[0].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [16]:
docs_ss[1].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

Note the difference in results with `MMR`.

In [17]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)

In [18]:
docs_mmr[0].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [19]:
docs_mmr[1].page_content[:100]

"mathematical work, he feels like he's disc overing truth and beauty in the universe. And \nhe says it"

### Addressing Specificity: working with metadata

In last lecture, we showed that a question about the third lecture can include results from other lectures as well.

To address this, many vectorstores support operations on `metadata`.

`metadata` provides context for each embedded chunk.

In [20]:
question = "what did they say about regression in the third lecture?"

In [35]:
IDS = vectordb._collection.peek()['ids']
IDS

['05e62b88-9108-459f-b0d1-e4afa79a00e3',
 '060b7504-6232-4137-971f-71a4830bf21d',
 '08f77f5d-f7cd-4c28-9734-03e434097c63',
 '099fdeeb-bdd5-4e8c-8d59-132e596d4be9',
 '09ed7880-1b8a-4f48-a1a9-adc43e6a8a0a',
 '0c7f63ba-0a54-4322-9295-3f967b97cf99',
 '0db3aa8e-9f6b-41c6-8766-91b9c90de279',
 '0e73eeea-9ed8-4f10-92d4-ce343fc362e6',
 '0f4d63dc-b1d4-4e41-9365-36d654822c9c',
 '10001f09-8289-49e9-acca-3a6b98bd792d']

In [37]:
vectordb._collection.get(IDS[0])

{'ids': ['05e62b88-9108-459f-b0d1-e4afa79a00e3'],
 'embeddings': None,
 'metadatas': [{'page': 6, 'source': 'data/MachineLearning-Lecture01.pdf'}],
 'documents': ["to happen.  \nThe section here on the late homework polic y if you ever want to hand in a homework \nlate, I'll leave you to r ead that yourself.  \nWe also have a midterm, which is scheduled for Thursday, 8th of November at 6:00 p.m., \nso please keep that evening free.  \nAnd let's see. And one more administrative thing I wanted to sa y is about the class \nproject. So part of the goal of this cla ss is to leave you well eq uipped to apply machine \nlearning algorithms to a problem or to do rese arch in machine learning. And so as part of \nthis class, I'll ask you to execute a small resear ch project sort of as a small term project.  \nAnd what most students do for this is either  apply machine learning to a problem that you \nfind interesting or investigate some aspect of  machine learning. So to those of you that \nare 

In [40]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":"data/MachineLearning-Lecture03.pdf"}
)

In [41]:
for d in docs:
    print(d.metadata)

{'page': 0, 'source': 'data/MachineLearning-Lecture03.pdf'}
{'page': 14, 'source': 'data/MachineLearning-Lecture03.pdf'}
{'page': 6, 'source': 'data/MachineLearning-Lecture03.pdf'}


### Addressing Specificity: working with metadata using self-query retriever

But we have an interesting challenge: we often want to infer the metadata from the query itself.

To address this, we can use `SelfQueryRetriever`, which uses an LLM to extract:
 
1. The `query` string to use for vector search
2. A metadata filter to pass in as well

Most vector databases support metadata filters, so this doesn't require any new databases or indexes.

In [43]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [44]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="""The lecture the chunk is from, should be one of 
        `data/MachineLearning-Lecture01.pdf`,
        `data/MachineLearning-Lecture02.pdf`, or
        `data/MachineLearning-Lecture03.pdf`""",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

**Note:** The default model for `OpenAI` ("from langchain.llms import OpenAI") is `text-davinci-003`. Due to the deprication of OpenAI's model `text-davinci-003` on 4 January 2024, you'll be using OpenAI's recommended replacement model `gpt-3.5-turbo-instruct` instead.

In [54]:
SelfQueryRetriever?

In [48]:
document_content_description = "Lecture notes"
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [49]:
question = "what did they say about regression in the third lecture?"

**You will receive a warning** about predict_and_parse being deprecated the first time you executing the next line. This can be safely ignored.

In [50]:
docs = retriever.get_relevant_documents(question)

In [51]:
for d in docs:
    print(d.metadata)

{'page': 2, 'source': 'data/MachineLearning-Lecture03.pdf'}
{'page': 10, 'source': 'data/MachineLearning-Lecture03.pdf'}
{'page': 11, 'source': 'data/MachineLearning-Lecture03.pdf'}
{'page': 6, 'source': 'data/MachineLearning-Lecture03.pdf'}


### Additional tricks: compression

Another approach for improving the quality of retrieved docs is compression.

Information most relevant to a query may be buried in a document with a lot of irrelevant text. 

Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this. 

In [55]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [56]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [57]:
# Wrap our vectorstore
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct")
compressor = LLMChainExtractor.from_llm(llm)

In [58]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [59]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

- "those homeworks will be done in either MATLA B or in Octave"
- "I know some people call it a free ve rsion of MATLAB"
- "MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data."
- "there's also a software package called Octave that you can download for free off the Internet."
- "it has somewhat fewer features than MATLAB, but it's free, and for the purposes of this class, it will work for just about everything."
- "once a colleague of mine at a different university, not at Stanford, actually teaches another machine learning course."
----------------------------------------------------------------------------------------------------
Document 2:

- "those homeworks will be done in either MATLA B or in Octave"
- "I know some people call it a free ve rsion of MATLAB"
- "MATLAB is I guess part of the programming language that makes it very easy to write 

In [60]:
len(compressed_docs)

4

## Combining various techniques

In [ ]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [ ]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

## Other types of retrieval

It's worth noting that vectordb as not the only kind of tool to retrieve documents. 

The `LangChain` retriever abstraction includes other ways to retrieve documents, such as TF-IDF or SVM.

In [71]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [72]:
# Load PDF
loader = PyPDFLoader("data/MachineLearning-Lecture01.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)

In [73]:
len(splits)

45

In [74]:
# Retrieve
svm_retriever = SVMRetriever.from_texts(splits, embedding)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

C:\Users\rauna\Anaconda3\envs\kaggle\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [75]:
question = "What are major topics for this class?"
docs_svm=svm_retriever.get_relevant_documents(question)
docs_svm[0]

Document(page_content="or find project partners or discuss homework problems and so on, and it's not monitored \nby the TAs and me. So feel free to ta lk trash about this class there.  \nIf you want to contact the teaching staff, pl ease use the email address written down here, \ncs229-qa@cs.stanford.edu. This goes to an acc ount that's read by all the TAs and me. So \nrather than sending us email individually, if you send email to this account, it will \nactually let us get back to you maximally quickly with answers to your questions.  \nIf you're asking questions about homework probl ems, please say in the subject line which \nassignment and which question the email refers to, since that will also help us to route \nyour question to the appropriate TA or to me  appropriately and get the response back to \nyou quickly.  \nLet's see. Skipping ahead — let's see — for homework, one midterm, one open and term \nproject. Notice on the honor code. So one thi ng that I think will help you to

In [78]:
print(docs_svm[0].page_content)

or find project partners or discuss homework problems and so on, and it's not monitored 
by the TAs and me. So feel free to ta lk trash about this class there.  
If you want to contact the teaching staff, pl ease use the email address written down here, 
cs229-qa@cs.stanford.edu. This goes to an acc ount that's read by all the TAs and me. So 
rather than sending us email individually, if you send email to this account, it will 
actually let us get back to you maximally quickly with answers to your questions.  
If you're asking questions about homework probl ems, please say in the subject line which 
assignment and which question the email refers to, since that will also help us to route 
your question to the appropriate TA or to me  appropriately and get the response back to 
you quickly.  
Let's see. Skipping ahead — let's see — for homework, one midterm, one open and term 
project. Notice on the honor code. So one thi ng that I think will help you to succeed and 
do well in this clas

In [76]:
question = "what did they say about matlab?"
docs_tfidf=tfidf_retriever.get_relevant_documents(question)
docs_tfidf[0]

Document(page_content="Saxena and Min Sun here did, wh ich is given an image like this, right? This is actually a \npicture taken of the Stanford campus. You can apply that sort of cl ustering algorithm and \ngroup the picture into regions. Let me actually blow that up so that you can see it more \nclearly. Okay. So in the middle, you see the lines sort of groupi ng the image together, \ngrouping the image into [inaudible] regions.  \nAnd what Ashutosh and Min did was they then  applied the learning algorithm to say can \nwe take this clustering and us e it to build a 3D model of the world? And so using the \nclustering, they then had a lear ning algorithm try to learn what the 3D structure of the \nworld looks like so that they could come up with a 3D model that you can sort of fly \nthrough, okay? Although many people used to th ink it's not possible to take a single \nimage and build a 3D model, but using a lear ning algorithm and that sort of clustering \nalgorithm is the first ste

In [79]:
print(docs_tfidf[0].page_content)

Saxena and Min Sun here did, wh ich is given an image like this, right? This is actually a 
picture taken of the Stanford campus. You can apply that sort of cl ustering algorithm and 
group the picture into regions. Let me actually blow that up so that you can see it more 
clearly. Okay. So in the middle, you see the lines sort of groupi ng the image together, 
grouping the image into [inaudible] regions.  
And what Ashutosh and Min did was they then  applied the learning algorithm to say can 
we take this clustering and us e it to build a 3D model of the world? And so using the 
clustering, they then had a lear ning algorithm try to learn what the 3D structure of the 
world looks like so that they could come up with a 3D model that you can sort of fly 
through, okay? Although many people used to th ink it's not possible to take a single 
image and build a 3D model, but using a lear ning algorithm and that sort of clustering 
algorithm is the first step. They were able to.  
I'll just 